The collective ultimate dank meme generator using ngram analysis. Crowd mentality is often found in cases where anonymity and brief exposure reign free in communicative environments. Using twitch chat data we will analyze for patterns of high repetition phrases such as "LUL", "Harambe", and "9/11 was an inside job" as well as other emoticons such as "Kappa" and "PogChamp". We will call these words "memes" and use a statistic "dankness" to measure how virulent these words are. Using an N-gram model we will generate subsequent "memes" based off the dankness statistic to generate the most likely to be used dank meme.

We use the twitch chat API to set up an IRC bot that sits in various channels that records the output of these channels. We decided to use these channels based off popularity; we intend to see a correlation between dankness and popularity of channel. These chat logs are massive in size since anonymity gives people leeway to post whatever they want whether it be offensive, politically charged, or pure nonsense. We note that in general, it is easier to convey a certain message with a shorter amount of words which can often lead to short non-sensical words that are generally accepted by the chat to have a certain meaning. We see that in the data, short phrases are favored over longer sentences. In fact, twitch has its own emoticons usually some sort of facial expression to indicate a message. One exception to the rule are "copy-pastas" which are long, pre-written paragraphs of text that are copy and pasted throughout chat, usually indicating satirical undertones although sometimes more direct. Chat will also react to things going on during the stream session. From this we can see clustering of chats being sent (which will be indicative from timestamps) around certain times usually of similar content.






In [ ]:
# Set up for methods and stuff

import datetime
import socket
import select
import re

channels = [
    'tsm_bjergsen',
    'c9sneaky',
    'trick2g',
    'tsm_doublelift',
    'nightblue3',
    'imaqtpie',
    'rush'
    ]
username = 'tacobff'
oauth = 'oauth:f9gp83pucfzriac4hjsfh5hwjdqho6'

channelfiles = {}

for name in channels:
    channelfiles[name] = open(name+'2.txt', 'a+')
#     channelfiles[name].write("\n \n------------ \n \n \n \n")
#     channelfiles[name].write("new session\n")
#     channelfiles[name].write("\n \n \n ------------ \n \n \n")

def ping():
    ''' Respond to the server 'pinging' (Stays connected) '''
    socks[0].send('PONG :pingis\n')
    print('PONG: Client > tmi.twitch.tv')

def sendmsg(chan,msg):
    ''' Send specified message to the channel '''
    socks[0].send('PRIVMSG '+chan+' :'+msg+'\n')
    print('[BOT] -> '+chan+': '+msg+'\n')

def sendwhis(user,msg):
    socks[1].send('PRIVMSG #jtv :/w '+user+' '+msg+'\n')
    print('[BOT] -> '+user+': '+msg+'\n')

def getmsg(msg):
    ''' GET IMPORTANT MESSAGE '''
    if(re.findall('@(.*).tmi.twitch.tv PRIVMSG (.*) :(.*)',msg)):
        msg_edit = msg.split(':',2)
        if(len(msg_edit) > 2):
            user = msg_edit[1].split('!',1)[0] # User
            message = msg_edit[2] # Message
            channel = re.findall('PRIVMSG (.*)',msg_edit[1]) # Channel
            #print channel[0][1:-1]
            privmsg = re.findall('@(.*).tmi.twitch.tv PRIVMSG (.*) :(.*)',msg)
            ''' CONVERT TO ARRAY '''
            privmsg = [x for xs in privmsg for x in xs]

            datelog = datetime.datetime.now()

            ''' PRINT TO CONSOLE '''
            if(len(privmsg) > 0):
                if len(channel) > 0:
                    chan = channel[0] 
                    if len(chan) >= 3:
    
                        #print ('['+str(datelog.hour)+':'+str(datelog.minute)+':'+str(datelog.second)+'] '+user+' @ '+channel[0][:-1]+': '+message+'\n')
                        channelfiles[channel[0][1:-1]].write('['+str(datelog.day)+':'+str(datelog.hour)+':'+str(datelog.minute)+':'+str(datelog.second)+'] '+user+' @ '+channel[0][:-1]+': '+message+'\n')
                
    if(re.findall('@(.*).tmi.twitch.tv WHISPER (.*) :(.*)',msg)):
        whisper = re.findall('@(.*).tmi.twitch.tv WHISPER (.*) :(.*)',msg)
        whisper = [x for xs in whisper for x in xs]

        ''' PRINT TO CONSOLE '''
        if(len(whisper) > 0):
            ''' PRINT WHISPER TO CONSOLE '''
            print('*WHISPER* '+whisper[0]+': '+whisper[2])
            

socks = [socket.socket(),socket.socket()]

socks[0].connect(('irc.twitch.tv',6667))

socks[0].send('PASS '+oauth+'\n')
socks[0].send('NICK '+username+'\n')

# socks[0].send("JOIN #tsm_bjergsen \r\n")

for val in channels:

    socks[0].send('JOIN #'+val+'\n')
    
print('Connected to irc.twitch.tv on port 6667')
print('USER: '+username)
print('OAUTH: oauth:'+'*'*30)
print('\n')

temp = 0
count = 0
while True:
  
    (sread,swrite,sexc) = select.select(socks,socks,[],120)
    for sock in sread:
  
        ''' Receive data from the server '''
        msg = sock.recv(2048)
        if(msg == ''):
            temp + 1
            if(temp > 5):
                print('Connection might have been terminated')
    
        ''' Remove any linebreaks from the message '''
        msg = msg.strip('\n\r')

        ''' DISPLAY MESSAGE IN SHELL '''
        getmsg(msg)
#         print(msg)


        # ANYTHING TO DO WITH CHAT FROM CHANNELS
        ''' GET THE INFO FROM THE SERVER '''
        check = re.findall('@(.*).tmi.twitch.tv PRIVMSG (.*) :(.*)',msg)
        if(len(check) > 0):
            msg_edit = msg.split(':',2)
            if(len(msg_edit) > 2):
                user = msg_edit[1].split('!',1)[0] # User
                message = msg_edit[2] # Message
                channel = msg_edit[1].split(' ',2)[2][:-1] # Channel

                msg_split = str.split(message)
 
                

                        
        # ANYTHING TO DO WITH WHISPERS RECIEVED FROM USERS
        check = re.findall('@(.*).tmi.twitch.tv WHISPER (.*) :(.*)',msg)
        if(len(check) > 0):
            msg_edit = msg.split(':',2)
            if(len(msg) > 2):
                user = msg_edit[1].split('!',1)[0] # User
                message = msg_edit[2] # Message
                channel = msg_edit[1].split(' ',2)[2][:-1] # Channel

                whis_split = str.split(message)
                               
 

        ''' Respond to server pings '''
        if msg.find('PING :') != -1:
            print('PING: tmi.twitch.tv > Client')
            ping()
print "stopped"

#https://github.com/RubbixCube/Twitch-Chat-Bot-V2

Connected to irc.twitch.tv on port 6667
USER: tacobff
OAUTH: oauth:******************************




## Preprocessing

In [39]:

channels = [
    'tsm_bjergsen',
    'c9sneaky',
    'trick2g',
    'tsm_doublelift',
    'nightblue3',
    'imaqtpie',
    'rush'
    ]

def parseTwitchChat(textpath):
    #returns an array of text
    text = []
    with open(textpath, 'r') as f:
        for line in f:
            text.append(line.split())
    count = 0
    newtext = []
    for line in text:
        
        if len(line) > 2:
            count += 1
            if line[1] == 'PRIVMSG':
                line = line[3:]
                
                line[0] = line[0][1:]
                newtext.append(line)
#                 if count <= 10:
#                     print line
            else:    
                line = line[4:]
                newtext.append(line)
#                 if count <= 10:
#                     print line


    return newtext

textFiles = {}
labels = []

for i in channels:
    textFiles[i] = (parseTwitchChat(i + ".txt"))
    labels.append(i)
    
for i in channels:
   
    for k in range(len(textFiles[i])): #for array inside textfile
        if (k % 1000) == 0:
            with open(i + '/league/' + i + '_' + str(k) +'_parsed.txt', 'a+') as f: #open every 1000 lines
                f.truncate()
                lenleftover = len(textFiles[i]) - k
                if lenleftover >= 1000:
                    lenleftover = 1000
                
                for j in range(lenleftover): #indivudal word in array
                    for l in textFiles[i][j+k]: #iterate through 1000 times for every 1000 lines
                        f.write(l +' ') #l = word
                    f.write('\n')



    
        

## Naive bayes classifier

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import load_files

#load files from very specific format check 
#http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html#sklearn-datasets-load-files
files_train = load_files('league')


#vectorize the data
count_vect = CountVectorizer()


#vector transform 

X_train_counts = count_vect.fit_transform(files_train.data)




tf_transformer = TfidfTransformer().fit(X_train_counts)

X_train_tfidf = tf_transformer.transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, files_train.target)

How mindless is twitch chat?

The average twitch user’s chat logs is made up of two kinds of messages: spam and the creation of messages for other users to spam. Because of the genuine lack of creativity and intelligence among the average twitch user’s chat logs, we hypothesize that a computer can emulate both kinds of messages. 

Emulating spam is trivial; often in twitch chat one message is spammed so often that it becomes difficult to see messages of any other kind. A computer would have no problem keeping track of the frequencies of messages so its an easy task for a computer.

Original messages are much more difficult to create. First, in order to try and emulate creative messages, we will establish a measure called dankness. We believe that a successful original message will be spammed more than unsuccessful ones, so dankness will be defined as the number of times a phrase was chatted. Due to the seemingly randomness nature of Twitch chat, we first approach this problem using an ngram model run on the Twitch chat data. We tried using a trigram model at first, but we found that a interpolated model with bigram and unigrams works a lot better due to the small average length of each message. 

In [5]:
import math
import os

"""

"""
def tokenize_by_word(sentence):
	return nltk.word_tokenize(sentence)
class NgramModel:
	"""
	__init__ initializes a NgramModel. 

	Parameters:
	n                       the n used in the ngram model
	train                   a string containing a training text
	unknown_replace_limit   if a word in the training text appears less than this many times, replace it with 'UNKNOWNWORD'
	"""
	def __init__(self, n, train, unknown_replace_limit=0):
        if (n < 0):
			raise Exception("N must be greater than or equal to zero to make an Ngram model.")

		tokens = tokenize_by_word(train)
		tokens_count = dict()
		for token in tokens:
			if token not in tokens_count:
				tokens_count[token] = 1
			else:
				tokens_count[token] += 1

		tokens = ["UNKNOWNWORD" if tokens_count[token] < unknown_replace_limit else token for token in tokens]


		ngrams = zip(*[tokens[i:] for i in range(n)])
		ngrams_count = dict()
		for ngram in ngrams:
			if ngram not in ngrams_count:
				ngrams_count[ngram] = 1
			else:
				ngrams_count[ngram] += 1

		n1grams = []
		n1grams_count = dict()
		if (n > 1):
			n1grams = zip(*[tokens[i:] for i in range(n - 1)])
			for n1gram in n1grams:
				if n1gram not in n1grams_count:
					n1grams_count[n1gram] = 1
				else:
					n1grams_count[n1gram] += 1

		self.n = n
		self.ngrams_count = ngrams_count
		self.n1grams_count = n1grams_count
		self.vocab_size = len(set(tokens))

	"""
	perplexity: string -> float

	Returns the perplexity of 'text' according to this language model.
	"""
	def perplexity(self, text):
		tokens = tokenizer.tokenize_by_word(text)
		if (len(tokens) < self.n):
				raise Exception("text does not have enough words to calculate perplexity")
		s = 0.0
		num = 0
		if (self. n > 0):
			for token in zip(*[tokens[i:] for i in range(self.n)]):
				prob = self.probability(token)

				if (prob == 0.0):
					prob = 0.00000000000000000000001

				s += math.log(prob)
				num += 1

			return math.exp(-s / num)
		else:
			for token in tokens:
				prob = self.probability(token)

				if (prob == 0.0):
					prob = 0.00000000000000000000001

				s += math.log(prob)
				num += 1

			return math.exp(-s / num)

	'''
	probability: tuple -> float

	probability takes in a n-length tuple representing a ngram and returns its probability
	according to this language model.
	'''
	def probability(self, ngram):
		prob = 0.0
		if (self.n > 1):
			n1gram = tuple(list(ngram)[0:(self.n - 1)])
			if ngram in self.ngrams_count and n1gram in self.n1grams_count:
				prob = float(self.ngrams_count[ngram]) / float(self.n1grams_count[n1gram])
		elif (self.n == 0):
			prob = float(1) / float(self.vocab_size)
		else:
			# This is for the unigram case
			if ngram in self.ngrams_count:
				total = 0
				for key in self.ngrams_count:
					total += self.ngrams_count[key]
				prob = float(self.ngrams_count[ngram]) / float(total)

		return prob




In [4]:
'''
InterpolatedModel: a class for representing an interpolated ngram model
'''
class InterpolatedModel:
	"""
	__init__ initializes an InterpolatedModel. 

	Parameters:
	models      the models used in the interpolation
	lambdas     the coefficients for each model
	"""
	def __init__(self, models, lambdas):
		if (not len(models) == len(lambdas)):
			raise Exception("The length of the models list must be the same as the length of the lambdas list")

		if (not sum(lambdas) == 1.0):
			raise Exception("The sum of the lambda coefficients must be 1")

		self.lambdas = lambdas
		self.models = models
		self.n = max([model.n for model in models])


	'''
	probability: tuple -> float

	probability takes in a tuple representing a ngram and returns its probability
	according to this language model. The tuple length must be less than or equal to n for
	the largest model.
	'''
	def probability(self, ngram):
		prob = 0.0
		for coefficient, model in zip(self.lambdas, self.models):
		 	n = model.n
		 	if (len(ngram) >= n):	
			 	word = tuple(list(ngram)[-n:])
			 	prob += coefficient * model.probability(word)
		return prob

	"""
	perplexity: string -> float

	Returns the perplexity of 'text' according to this language model.
	"""
	def perplexity(self, text):
		tokens = tokenizer.tokenize_by_word(text)
		s = 0.0
		num = 0
		for i in range(0, len(tokens)):
			if (i < self.n):
				prob = self.probability(tuple(tokens[0:i + 1]))
			else:
				prob = self.probability(tuple(tokens[i - self.n + 1:i + 1]))

			if (prob == 0.0):
				prob = 0.00000000000000000000001
			s += math.log(prob)
			num += 1
		return math.exp(-s / num)

As of now, we have gathered a lot of chat data from multiple different streamers and stored them separately. 